# 積分計算

$$
    \pi = \int_0^1 \frac{4}{1+x^2} dx
$$

In [1]:
versioninfo()

Julia Version 1.10.0
Commit 3120989f39b (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M2
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
  Threads: 5 on 4 virtual cores
Environment:
  JULIA_NUM_THREADS = 4


In [2]:
using IntervalArithmetic
using Pkg; Pkg.status("IntervalArithmetic")

Status `~/.julia/environments/v1.10/Project.toml`
⌃ [d1acc4aa] IntervalArithmetic v0.22.8
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [3]:
include("IntervalCheb.jl")

chebmax (generic function with 2 methods)

被積分関数 $f(x) = 4/(1+x^2)$ を定義

In [4]:
f(x) = 4 / (1+x^2)
f(x::Interval{T}) where {T<:Real} = interval(4)/(interval(1) + x^2) # Define f when the input is a interval type
f(x::Complex{Interval{T}}) where {T<:Real} = interval(4)/(interval(1) + x*x) # Define f when the input is a complex interval type

f (generic function with 3 methods)

区間 $[0,1]$ において区間チェビシェフ補間を構成する。

In [5]:
dom = [0,1]
ip = interval_cheb(f,dom)

23-element Vector{Interval{Float64}}:
 [3.10754, 3.10755]_com
 [-1.06635, -1.06634]_com
 [-0.100688, -0.100687]_com
 [0.0682289, 0.068229]_com
 [-0.00752218, -0.00752217]_com
 [-0.00185697, -0.00185696]_com
 [0.000687293, 0.000687294]_com
 [-3.61477e-05, -3.61476e-05]_com
 [-2.58243e-05, -2.58242e-05]_com
 [6.33881e-06, 6.33882e-06]_com
   ⋮
 [-3.40586e-09, -3.40585e-09]_com
 [3.65637e-10, 3.6564e-10]_com
 [9.44655e-11, 9.44674e-11]_com
 [-3.41643e-11, -3.41616e-11]_com
 [1.69401e-12, 1.69603e-12]_com
 [1.30067e-12, 1.30312e-12]_com
 [-3.14619e-13, -3.12619e-13]_com
 [-6.17217e-15, -3.63348e-15]_com
 [1.45715e-14, 1.67588e-14]_com

In [6]:
maximum(radius,ip)

2.353672812205332e-14

区間 $[0,1]$ において定積分を計算する。`iPi` が積分値の包含になっており、その区間半径は約 $1.8\cdot 10^{-14}$ となる。

In [7]:
iPi = chebint(ip,dom)
radius(iPi)

2.6645352591003757e-14

In [8]:
include("IntervalCheb.jl")
function pi_int()
    dom = [0, 1]
    f(x) = 4 / (1+x^2)
    f(x::Complex{Interval{T}}) where {T} = interval(4)/(interval(1) + x*x)
    f(x::Interval{T}) where {T} = interval(4)/(interval(1) + x^2)
    # ia = interval_cheb(f,interval(2^-52),2^-3)
    # return chebint(ia) / interval(2)
    return chebint(interval_cheb(f,dom),dom)
end

pi_int (generic function with 1 method)

全ての過程を一つの関数としてまとめて、実行時間を計測する。

In [9]:
@time pi_int()
@time pi_int()
@time pi_int()
@time pi_int()
@time iPi = pi_int()

  0.046831 seconds (42.94 k allocations: 3.040 MiB, 217.43% compilation time)
  0.000282 seconds (724 allocations: 177.172 KiB)
  0.000249 seconds (724 allocations: 177.422 KiB)
  0.000268 seconds (723 allocations: 177.062 KiB)
  0.000228 seconds (724 allocations: 177.094 KiB)


[3.14159, 3.1416]_com

In [10]:
radius(pi_int())

2.6645352591003757e-14

計算して得られた積分値 `iPi` は $\pi$ を包含していることが確認できる。

In [11]:
issubset_interval(interval(π),iPi)
# @show bitstring(sup(interval(π)))
# @show bitstring(inf(interval(π)))

true

In [12]:
radius(interval(π))

4.440892098500626e-16